# CMS Pricing Analytics
------------------
> **Idriss Afra**

This project aims to compute the CMS convexity adjustment and price CMS options.

## The Model Description

CMS (Constant Maturity Swap) derivatives are widely used because they offer an efficient way to gain exposure to long-term interest rates. The payoff of a CMS product depends on a swap rate $S$ with a fixed maturity $T_f$ paid at $T_p$, and is typically written as : $f\left(S(T_f, T_0, T_1)\right)$, where :

* $f$ is the payoff function. For example, $f(s)=s$ represents a CMS forward.
* $S$ is the swap rate.
* $T_f \ge 0$ is the fixing date, when the swap rate is observed.
* $T_0$ is the start date of the swap. Usually, $T_0 = T_f + 2BD$.
* $T_1$ is the end date, making $T_1 - T_0$ the tenor (length) of the swap.

Assume a cash-flow of $f\left(S(T_f, T_0, T_1)\right)$ paid at $T_p  \in [T_0, T_1]$. Its present value $PV$ under the $T_p$-forward measure $Q^{T_p}$ is :

$$
PV = B(0, Tp) \times E^{Q^{T_p}} \left(f\left(S(T_f, T_0, T_1)\right)\right)
$$

Where : $B(0, Tp)$ is the zero-coupon price up to $T_p$.

To make pricing easier, we change the measure from $Q^{T_p}$ to the level measure $Q^{LVL}$, under which the swap rate $S(t, T_0, T_1)$ behaves like a martingale (i.e., has no drift). Using the Radon-Nikodym change of measure, the PV becomes :   

\begin{equation}
\begin{split}
PV & = B(0, Tp) \times E^{Q^{T_p}} \left( \frac{f\left(S(T_f, T_0, T_1)\right) \times B(T_f, T_p) }{B(T_f, T_p)} \right) \\
& = LVL(0,T_0, T_1) \times E^{Q^{LVL}} \left( \frac{f\left(S(T_f, T_0, T_1)\right) \times B(T_f, T_p) }{LVL(T_f,T_0, T_1)} \right) \\
\end{split}
\end{equation}

Where : $LVL(t,T_0, T_1) = ∑_{T^p_i \in FixedLegPayDates} DCF_i × B(T_f, T^p_{i})$, with $(DCF_i)_{T^p_i \in FixedLegPayDates}$ being the fixed leg day count fractions.

## The TSR Model : Linear Mean-Reversion

In the Terminal Swap Rate (TSR) model, we assume that the zero-coupon bond prices $\left(B(T_f, T^p_i)\right)_{T^p_i \in FixedLegPayDates}$, along with $B(T_f, T_p)$, are directly related to the swap rate $S(T_f, T_0, T_1)$ that drives the CMS payoff.

This means there exists a function $g$ such that : $ \frac{B(T_f, T_p)}{LVL(T_f,T_0, T_1)} ≈ g\left(S(T_f, T_0, T_1)\right) $

In this project, we use the Linear Mean-Reversion TSR method, where : $g(s) := a \times s + b$.

### Calculating the Slope a

The coefficient $a$ is defined by :
\begin{equation}
\begin{split}
a & = \frac{d}{dS(T_f, T_0, T_1)} \left( \frac{B(T_f, T_p)}{LVL(T_f,T_0, T_1)}  \right) \\
& = \frac{d}{dS(T_f)} \left( \frac{B(T_f, T_p)}{∑_{T^p_i \in FixedLegPayDates} DCF_i × B(T_f, T^p_{i}) } \right) \\
\end{split}
\end{equation}

We rewrite this derivative in the context of the [Hull-White One Factor model](https://github.com/Idriss-Afra/Hull-White-One-Factor-Model/blob/main/Hull-White%201F%20Model.ipynb) as :    

$$
a = \frac{d}{dX_{T_f}} \left( \frac{B(T_f, T_p, X_{T_f})}{∑_{T^p_i \in FixedLegPayDates} DCF_i × B(T_f, T^p_{i}, X_{T_f})}  \right)_{S(T_f, X_{T_f}) = S(0)} × \frac{1}{\left( \frac{d S(T_f, X_{T_f})}{dX_{T_f}}\right)_{S(T_f, X_{T_f}) = S(0)}}
$$

Where :    
* $X_t$ is the Hull-White one factor state variable.
* $B(t, T, X_t)$ is the Hull-White one factor zero-coupon log-normal price.
* $S(0)$ is the forward swap rate.

We use the following approximation : $ B(T_f, T, X_{T_f})_{S(T_f, X_{T_f}) = S(0)} ≈ \frac{B(0, T)}{B(0, T_f)} $, for $T \in ]T_0, T_1]$.

Thus, the slope becomes :    

$$
a = \frac{B(0, T_p) × (γ - \beta(T_f, T_p))}{B(0, T_N)×β(T_f, T_N)+LVL(0, T_0, T_1)×S(0, T_0, T_1)×γ}
$$

Where :

$$
γ = \frac{∑_{T^p_i \in FixedLegPayDates} DCF_i × B(0, T^p_{i})×β(T_f, T^p_{i})}{LVL(0, T_0, T_1)}
$$

### Calculating the Intercept b

On the other hand, for $T_f = 0$ :    

$$
b = \frac{B(0, T_p)}{LVL(0,T_0, T_1)} - a × S(0, T_0, T_1)
$$

We first implement a zero-coupon curve class to define discounting curves :

In [1]:
import numpy as np
from scipy.interpolate import interp1d

class zc_curve :
    """
    Zero-Coupon Rates Curve class.
    """
    def __init__(self, maturities, zc_rates):
        """
        Init method : Takes the IR market tenors and the bootstrapped ZC rates.
        """
        self.maturities = maturities
        self.zc_rates = zc_rates
        # Cubic Interpolation & Extrapolation
        self.zc_rates_interp = interp1d(maturities, zc_rates, kind='cubic', fill_value="extrapolate")

    def df(self, T):
        """
        Zero-Coupon Discount Factors.
        """
        return np.exp(-self.zc_rates_interp(T)*T)

Then, we implement the TSR model as described above :

In [2]:
class tsr_model :
    """
    TSR model : The Linear Mean-Reversion method class.
    """
    def __init__(self, df, mean_reversion=0.02) :
        """
        Init method : df is the discount factor function and mean_reversion is the HW-1F mean-reversion (defaulted to 2%).
        """
        self.df = df
        self.mean_reversion = mean_reversion

    def beta(self, t, T) :
        """
        HW-1F Beta function.
        """
        return (1 - np.exp(-self.mean_reversion * (T - t))) / self.mean_reversion

    def tsr_coeffs(self, expiry, tenor, pay_date) :
        """
        TSR coefficients formulas.
        """
        
        # Special Case : If the expiry is reached
        if expiry == 0:
            print("The input expiry is 0 : The TSR model is not applied.")
            return None
        
        # Dates
        start_time = expiry + 2./365.
        # Fixed leg payment dates : Annually
        pay_times = np.arange(start_time + 1, expiry + tenor + 1)
        year_fractions = np.ones(len(pay_times))

        # Intermidiate Variables
        df_start_time = self.df(start_time)
        df_pay_times = self.df(pay_times)
        level = np.sum(df_pay_times * year_fractions)
        swap_fwd = (df_start_time - df_pay_times[-1]) / level
        gamma = np.sum(df_pay_times * year_fractions * self.beta(expiry, pay_times)) / level

        # Results
        num = self.df(pay_date) * (gamma - self.beta(expiry, pay_date))
        den = df_pay_times[-1] * self.beta(expiry, pay_times[-1]) + level * swap_fwd * gamma
        a = num / den
        b = self.df(pay_date) / level - a * swap_fwd

        return {"a" : a, "b" : b}

## The Replication Method

To determine the present value (PV) of a CMS-linked product, we use the following formula derived earlier :

\begin{equation}
\begin{split}
PV & = LVL(0,T_0, T_1) \times E^{Q^{LVL}} \left( \frac{f\left(S(T_f, T_0, T_1)\right) \times B(T_f, T_p) }{LVL(T_f,T_0, T_1)} \right) \\
& ≈ LVL(0,T_0, T_1) \times E^{Q^{LVL}} \left[ f(S(T_f, T_0, T_1)) × g(S(T_f, T_0, T_1) \right] \\
& = LVL(0,T_0, T_1) \times E^{Q^{LVL}} \left[ h(S(T_f, T_0, T_1))  \right] \\
\end{split}
\end{equation}

Where : $h(s) = f(s) \times g(s)$

To evaluate the expected value, we use the Carr-Madan replication approach :

$$
h(S(T_f)) = h(K^*) + h'(K^*) × \left(S(T_f) - K^*\right) + ∫_{-∞}^{K^*}h''(k) \times (k-S(T_f))^+dk + ∫_{K^*}^{+∞}h''(k) \times (S(T_f) - k)^+dk
$$

Where : $K^*$ is generally chosen as the forward swap rate $S(0, T_0, T_1)$.

### CMS Forward Price

For CMS forwards, the payoff function is : $h(s) = s × (as + b) = as^2 + bs$.

So :
* $h'(s) = 2as + b$
* $h''(s) = 2a$

Thus, the value of a CMS forward is:

\begin{equation}
\begin{split}
CMS_0(T_f, T_0, T_1, T_p) & = \frac{PV}{B(0, T_p)}\\
&= \frac{LVL(0, T_0, T_1)}{B(0, T_p)} \left( \left(aS(0, T_0, T_1)^2+bS(0, T_0, T_1)\right) + ∫_{-∞}^{S(0, T_0, T_1)}2a \times Put_{Swaption}^{OTM}(T_f, k)dk \\+ ∫_{S(0, T_0, T_1)}^{+∞}2a \times Call_{Swaption}^{OTM}(T_f, k)dk \right) \\
\end{split}
\end{equation}

It is priced by a replication of out-of-the-money (OTM) swaptions and is higher than the forward swap rate $S(t, T_0, T_1)$, making it subject to a positive convexity adjustment.

### CMS Option Price

For a CMS caplet with strike $K$, the payoff function is : $h(s) = (s-K)^+ × (as + b)$.

So :
* $h'(s) = (2as + b - aK) × 1_{s > K}$
* $h''(s) = 2a × 1_{s > K}$

Because of the non-smoothness at the strike, the replication formula is adjusted as following :

$$
PV = LVL(0,T_0, T_1) \times \left( \left(aS(0, T_0, T_1)+b\right) \times (S(0, T_0, T_1) - K)^+ + \left(aK+b\right) × \\   \left( 1_{K<S(0, T_0, T_1)} \times Put_{Swaption}(T_f, K) + 1_{K \ge S(0, T_0, T_1)} \times Call_{Swaption}(T_f, K) \right) + \\ 
∫_{min(K, S(0, T_0, T_1))}^{S(0, T_0, T_1)}2a \times Put_{Swaption}^{OTM}(T_f, k)dk + \\
∫_{max(K, S(0, T_0, T_1))}^{+∞}2a \times Call_{Swaption}^{OTM}(T_f, k)dk \right)
$$

CMS floorlets can then be implied using the Call / Put parity... Similar to the CMS forward, the valuation relies on a replication by OTM swaptions.

We start by implementing the swaption instrument class :

In [3]:
import numpy as np
from scipy.stats import norm
import math

class swaption :
    """
    Vanilla instrument "Swaption" class.
    """
    def __init__(self, payer_receiver, expiry, tenor, strike, notional = 1.):
        """
        Init method.
        """
        self.payer_receiver = payer_receiver
        self.expiry = expiry
        self.strike = strike
        self.notional = notional
        self.start_time = expiry + 2./365.
        # Fixed leg payment dates : Annually
        self.pay_times = np.arange(self.start_time + 1, expiry + tenor + 1)
        self.year_fractions = np.ones(len(self.pay_times))

    def set_market_data(self, df, normal_vol):
        """
        Stores the DFs values to save computation time.
        """
        self.df_exp_time = df(self.expiry)
        self.df_start_time = df(self.start_time)
        self.df_pay_times = df(self.pay_times)
        self.normal_vol = normal_vol

    def level(self) :
      """
      The level function : Sum of DCF(T(i)) * B(0, T(i)) where T(i) are the fixed leg pay dates.
      """
      return np.sum(self.df_pay_times * self.year_fractions)

    def forward(self):
        """
        The forward level of the swap rate.
        """
        return (self.df_start_time - self.df_pay_times[-1]) / self.level()

    def pv_underlying(self):
        """
        PV of the underlying swap.
        """
        phi = 1. if (self.payer_receiver.upper()=='PAYER') else -1.
        return phi * self.notional * self.level() * (self.forward() - self.strike)

    def market_price(self):
        """
        Swaption price under the Normal model.
        """
        phi = 1 if (self.payer_receiver.upper()=='PAYER') else -1
        lvl = self.level()
        fwd = self.forward()
        if (self.expiry==0 or self.normal_vol==0) : return max(phi * (fwd - self.strike), 0)
        sqrt_V2T = self.normal_vol * math.sqrt(self.expiry)
        d = (fwd - self.strike) / sqrt_V2T
        return self.notional * lvl * sqrt_V2T * (phi * d * norm.cdf(phi * d) + norm.pdf(d))

    def market_fwd_price(self):
        """
        Swaption forward price under the Normal model.
        """
        phi = 1 if (self.payer_receiver.upper()=='PAYER') else -1
        lvl = self.level()
        fwd = self.forward()
        if (self.expiry==0 or self.normal_vol==0) : return max(phi * (fwd - self.strike), 0)
        sqrt_V2T = self.normal_vol * math.sqrt(self.expiry)
        d = (fwd - self.strike) / sqrt_V2T
        return self.notional * sqrt_V2T * (phi * d * norm.cdf(phi * d) + norm.pdf(d))

Next, we define a volatility skew class to construct normal vol skews through interpolation and extrapolation methods :

In [4]:
class vol_skew :
    """
    Normal Vol Skew
    """
    def __init__(self, strikes, vol_data) :
        """
        Init method : Takes the market market strikes and their implied normal volatilities.
        """
        self.strikes = strikes
        self.vol_data = vol_data

    def normal_vols_skew(self, strike) :
        """
        A function that takes the strike as input, and output the equivalent normal_vol(expiry, strike).
        Cubic interpolation & Linear extrapolation.
        """
        if strike < self.strikes[0] or strike > self.strikes[-1] :
            return interp1d(self.strikes, self.vol_data, kind='linear', fill_value="extrapolate")(strike)
        return interp1d(self.strikes, self.vol_data, kind='cubic')(strike)
    

Finally, we implement the Carr-Madan replication approach to price CMS forwards and options :

In [5]:
from scipy.integrate import quad

class replication_method :
    """
    CMS forwards and CMS Caplets / Floorlets prices using the Carr-Madan replication by OTM Swaptions.
    """
    def __init__(self, tsr_model) :
        """
        Init method : Takes the TSR model as input.
        """
        self.tsr_model = tsr_model

    def replication_price(self, payoff_type, expiry, tenor, pay_date, vol_skew, strike = 0., n_stdev = 5.):
        """
        payoff_type : Forward, Caplet, or Floorlet.
        vol_skew : A function that takes the strike as input, and output the equivalent normal_vol(expiry, strike).
        """
        # df
        df = self.tsr_model.df

        # Dates
        start_time = expiry + 2./365.
        pay_times = np.arange(start_time + 1, expiry + tenor + 1)
        year_fractions = np.ones(len(pay_times))

        # Intermidiate Variables
        df_start_time = df(start_time)
        df_pay_times = df(pay_times)
        df_pay_date = df(pay_date)
        level = np.sum(df_pay_times * year_fractions)
        swap_fwd = (df_start_time - df_pay_times[-1]) / level
        
        # If the expiry is reached
        if expiry == 0:
            if payoff_type.upper()=="FORWARD" :
                return df_pay_date * swap_fwd
            elif payoff_type.upper()=="CAPLET" :
                return df_pay_date * max(swap_fwd - strike, 0)
            elif payoff_type.upper()=="FLOORLET" :
                return df_pay_date * max(strike - swap_fwd, 0)
            else :
                print("Undefined PayOff type. Possible values : Forward, Caplet, or Floorlet.")
                return None
                
        # TSR coefficients
        tsr_coeffs = self.tsr_model.tsr_coeffs(expiry, tenor, pay_date)
        a = tsr_coeffs["a"]
        b = tsr_coeffs["b"]

        # Swaption Prices
        swopt_pay = swaption("Payer", expiry, tenor, swap_fwd)
        swopt_pay.set_market_data(df, vol_skew(swap_fwd))
        swopt_rec = swaption("Receiver", expiry, tenor, swap_fwd)
        swopt_rec.set_market_data(df, vol_skew(swap_fwd))
        def swopt_fwd_price(payer_receiver, k) :
            if payer_receiver.upper() == "PAYER" :
                swopt_pay.strike = k
                swopt_pay.normal_vol = vol_skew(k)
                return swopt_pay.market_fwd_price()
            else :
                swopt_rec.strike = k
                swopt_rec.normal_vol = vol_skew(k)
                return swopt_rec.market_fwd_price()

        # PayOff CMS Fwd functions
        payoff = lambda s : a * s**2 + b * s
        payoff_d = lambda s : 2 * a * s + b
        payoff_sd = lambda s : 2 * a

        # Intermidiate Functions : OTM Swaptions called in the integration
        otm_put_part = lambda k : payoff_sd(k) * swopt_fwd_price("Receiver", k)
        otm_call_part = lambda k : payoff_sd(k) * swopt_fwd_price("Payer", k)

        # CMS Fwd price by Replication :
        upper_bound = swap_fwd + n_stdev * vol_skew(swap_fwd) * math.sqrt(expiry)
        lower_bound = swap_fwd - n_stdev * vol_skew(swap_fwd) * math.sqrt(expiry)
        rep_price = payoff(swap_fwd) + quad(otm_put_part, lower_bound, swap_fwd)[0] + \
                    quad(otm_call_part, swap_fwd, upper_bound)[0]
        cms_fwd = level * rep_price / df_pay_date

        if payoff_type.upper()=="FORWARD" :
            print({"Swap Fwd" : round(100 * swap_fwd, 4), "CMS Fwd" : round(100 * cms_fwd, 4), 
                   "Conv. Adj." : round(100 * (cms_fwd - swap_fwd), 4)})
            return cms_fwd
        
        elif payoff_type.upper()=="CAPLET" or payoff_type.upper()=="FLOORLET" :
            # PayOff CMS Caplet functions
            g = lambda s : a * s + b
            payoff = lambda s : (a * s + b) * (s - strike)
            payoff_d = lambda s : 2 * a * s + b - a * strike
            payoff_sd = lambda s : 2 * a

            # CMS Caplet price by Replication
            rep_price = g(swap_fwd) * max(swap_fwd - strike, 0) + g(strike) * \
                        swopt_fwd_price("Receiver" if strike < swap_fwd else "Payer", strike) + \
                        quad(otm_put_part, min(strike, swap_fwd), swap_fwd)[0] + \
                        quad(otm_call_part, max(strike, swap_fwd), upper_bound)[0]

            if payoff_type.upper()=="CAPLET" :
                return level * rep_price
            else :
                # Call/Put Parity : CMS Floorlet price by Replication
                return level * rep_price - df_pay_date * (cms_fwd - strike)
        
        else :
            print("Undefined PayOff type. Possible values : Forward, Caplet, or Floorlet.")
            return None

## Numerical Application

Let us compute 10Y EUR CMS forwards and options fixed in 5Y and paid in 6Y, as of 1st Febrary 2024 :

In [6]:
# IR yield curve asof 1st Febrary 2024 (EURIBOR 6M)
maturities = np.array([0.5, 1, 2, 5, 6, 8, 10, 15, 20, 30])
zc_rates = np.array([3.84, 3.41, 2.84, 2.48, 2.47, 2.49, 2.52, 2.6, 2.53, 2.28]) / 100
yc = zc_curve(maturities, zc_rates)

# Normal Market Volatilities asof 1st Febrary 2024 (EURIBOR 6M) : Expiry 5Y x Tenor 10Y
expiry = 5.
tenor = 10.
pay_date = 6.
strikes_5y10y = np.array([1.18, 1.68, 2.18, 2.68, 3.68, 4.68, 5.18]) / 100.
normal_vols_5y10y = np.array([84.70, 83.81, 83.76, 84.74, 89.82, 98.07, 102.91]) / 10000.
vol_skew_5y10y = lambda k : vol_skew(strikes_5y10y, normal_vols_5y10y).normal_vols_skew(k)

# TSR Model : Swap 5Yx10Y to be paid in 6Y. We assume a Mean-Reversion of 1.50%.
mean_reversion = 0.015
tsr = tsr_model(yc.df, mean_reversion)

# CMS Convexity Adjustment : Swap 5Yx10Y to be paid in 6Y
print("5Yx10Y CMS Forward paid in 6Y (%) :")
print("-----------------------------------")
rep_method = replication_method(tsr)
cms_fwd = rep_method.replication_price("Forward", expiry, tenor, pay_date, vol_skew_5y10y)

5Yx10Y CMS Forward paid in 6Y (%) :
-----------------------------------
{'Swap Fwd': 2.6873, 'CMS Fwd': 2.8708, 'Conv. Adj.': 0.1835}


In [7]:
# ATM 5Yx10Y CMS Caplet & Floorlet with a payment date of 6Y
print("The ATM 5Yx10Y CMS Caplet & Floorlet paid in 6Y :")
print("-------------------------------------------------")
print("Caplet price (Bps) : ", round(10000. * 
                            rep_method.replication_price("Caplet", expiry, tenor, pay_date, vol_skew_5y10y, cms_fwd), 4))
print("Floorlet price (Bps) : ", round(10000. * 
                            rep_method.replication_price("Floorlet", expiry, tenor, pay_date, vol_skew_5y10y, cms_fwd), 4))

The ATM 5Yx10Y CMS Caplet & Floorlet paid in 6Y :
-------------------------------------------------
Caplet price (Bps) :  66.7931
Floorlet price (Bps) :  66.7931


In [8]:
# 5Yx10Y CMS Caplet & Floorlet with a payment date of 6Y and a strike != ATM
strike = 0.025
print("The 5Yx10Y CMS Caplet & Floorlet paid in 6Y with a strike of {} :".format(strike))
print("-------------------------------------------------------------------")
print("Caplet price (Bps) : ", round(10000. * 
                            rep_method.replication_price("Caplet", expiry, tenor, pay_date, vol_skew_5y10y, strike), 4))
print("Floorlet price (Bps) : ", round(10000. * 
                            rep_method.replication_price("Floorlet", expiry, tenor, pay_date, vol_skew_5y10y, strike), 4))

The 5Yx10Y CMS Caplet & Floorlet paid in 6Y with a strike of 0.025 :
-------------------------------------------------------------------
Caplet price (Bps) :  82.9371
Floorlet price (Bps) :  50.9655
